In [6]:
import torch
import torch.nn as nn
from model import Transformer
from config import get_config, get_weights_file_path
from train import get_model, get_ds, greedy_decode
import altair as alt
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [7]:
# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [8]:
from config import get_config
from train import get_ds, get_model
config = get_config()
train_dataloader, val_dataloader, vocab_src, vocab_tgt = get_ds(config)
model = get_model(config, vocab_src.get_vocab_size(), vocab_tgt.get_vocab_size()).to(device)



Max length of source sentence: 64
Max length of target sentence: 53


In [14]:
# Load the pretrained weights
# model_filename = get_weights_file_path(config, f"29")
model_filename = "runs/vi-en-translation/model:epoch=19-step:step=14300-train_loss:train_loss=1.52-val_loss:val_loss=1.51.ckpt"
state = torch.load(model_filename)
print(state.keys())


dict_keys(['epoch', 'global_step', 'pytorch-lightning_version', 'state_dict', 'loops', 'callbacks', 'optimizer_states', 'lr_schedulers'])


In [15]:
model.load_state_dict(state['state_dict'])

<All keys matched successfully>

In [16]:
def load_next_batch():
    # Load a sample batch from the validation set
    batch = next(iter(val_dataloader))
    encoder_input = batch["encoder_input"].to(device)
    encoder_mask = batch["encoder_mask"].to(device)
    decoder_input = batch["decoder_input"].to(device)
    decoder_mask = batch["decoder_mask"].to(device)

    encoder_input_tokens = [vocab_src.id_to_token(idx) for idx in encoder_input[0].cpu().numpy()]
    decoder_input_tokens = [vocab_tgt.id_to_token(idx) for idx in decoder_input[0].cpu().numpy()]

    # check that the batch size is 1
    assert encoder_input.size(
        0) == 1, "Batch size must be 1 for validation"

    model_out = greedy_decode(
        model, encoder_input, encoder_mask, vocab_src, vocab_tgt, config['seq_len'], device)
    
    return batch, encoder_input_tokens, decoder_input_tokens

In [20]:
def mtx2df(m, max_row, max_col, row_tokens, col_tokens):
    return pd.DataFrame(
        [
            (
                r,
                c,
                float(m[r, c]),
                "%.3d %s" % (r, row_tokens[r] if len(row_tokens) > r else "<blank>"),
                "%.3d %s" % (c, col_tokens[c] if len(col_tokens) > c else "<blank>"),
            )
            for r in range(m.shape[0])
            for c in range(m.shape[1])
            if r < max_row and c < max_col
        ],
        columns=["row", "column", "value", "row_token", "col_token"],
    )

def get_attn_map(attn_type: str, layer: int, head: int):
    if attn_type == "encoder":
        attn = model.encoder.layers[layer].self_attention_block.attention_scores
    elif attn_type == "decoder":
        attn = model.decoder.layers[layer].self_attention_block.attention_scores
    elif attn_type == "encoder-decoder":
        attn = model.decoder.layers[layer].cross_attention_block.attention_scores
    return attn[0, head].data

def attn_map(attn_type, layer, head, row_tokens, col_tokens, max_sentence_len):
    df = mtx2df(
        get_attn_map(attn_type, layer, head),
        max_sentence_len,
        max_sentence_len,
        row_tokens,
        col_tokens,
    )
    return (
        alt.Chart(data=df)
        .mark_rect()
        .encode(
            x=alt.X("col_token", axis=alt.Axis(title="")),
            y=alt.Y("row_token", axis=alt.Axis(title="")),
            color="value",
            tooltip=["row", "column", "value", "row_token", "col_token"],
        )
        #.title(f"Layer {layer} Head {head}")
        .properties(height=200, width=200, title=f"Layer {layer} Head {head}")
        .interactive()
    )

def get_all_attention_maps(attn_type: str, layers: list[int], heads: list[int], row_tokens: list, col_tokens, max_sentence_len: int):
    charts = []
    for layer in layers:
        rowCharts = []
        for head in heads:
            rowCharts.append(attn_map(attn_type, layer, head, row_tokens, col_tokens, max_sentence_len))
        charts.append(alt.hconcat(*rowCharts))
    return alt.vconcat(*charts)

In [18]:
batch, encoder_input_tokens, decoder_input_tokens = load_next_batch()
print(f'Source: {batch["src_text"][0]}')
print(f'Target: {batch["tgt_text"][0]}')
sentence_len = encoder_input_tokens.index("[PAD]")

Source: tôi luôn có thể đi xe buýt trở lại boston, phải không?
Target: I can always take the bus back to Boston, right?


# Nhận xét:
- Layer 0 Head 0 'xe' với 'buýt' có độ tương đồng cao
- Nhìn chung, cả layer 0 và ở head 0,1,2,3 và head 7, các từ đều chỉ tập trung vào chính nó (xem đường chéo chính)
- Nhìn chung ở layer 0, 'đi', 'xe' và 'trở' có độ tương đồng cao
- Layer 0 Head 2, 'Boston' nhận được attention từ rất nhiều từ ("xe","buýt","trở","lại",...) --> mô hình hiểu được việc "trở lại"  là sử dụng phương tiện i.e: "đi xe buýt" ...?
- Layer 2 Head 1, 'Boston' cũng nhận được rất nhiều attention
- Layer 1 Head 0,1,2,3 attention vùng "phải","không" và "?" tầm 0.1 trở lên --> hiểu được là câu hỏi?
- Layer 1 Head 1,2 "trở" "lại" "Boson" --> tương tự Layer 0 Head 2
- Layer 2 Head 7 "?" nhận được attention cao từ "phải", "không" --> hiểu là câu hỏi
- Token "EOS" Nhận nhiều attention của các token khác (Layer 2 Head 4, 6) --> vì đây là encoder --> attention được dồn vào token này?

In [21]:
layers = [0, 1, 2]
heads = [0, 1, 2, 3, 4, 5, 6, 7]

# Encoder Self-Attention
get_all_attention_maps("encoder", layers, heads, encoder_input_tokens, encoder_input_tokens, min(20, sentence_len))


alt.VConcatChart(...)

# Nhận xét attention giữa decoder với decoder
- Như decoder, nhìn chung, cả layer 0 và ở head 0,1,2,3 và head 7, các từ đều chỉ tập trung vào chính nó (xem đường chéo chính)
- Layer 0, Head 1, đại từ "I" tập trung vào các từ "I", "can", và "always" --> chủ ngữ của câu liên đến chính và động từ thiếu khuyết và tần xuất từ;
- Layer 0 Head 2 và 1 Head 2 tập trung vào , 'take' tập trung vào 'take', 'the' và 'bus' --> động từ liên quan đến việc bắt xe buýt;
- Layer 0 Head 6 "can", và "always" attention vào "take" --> giống Layer 0, Head 1 --> ...
- Layer 1 Head 4 "take", "back" , "to" attention vào "back"
- Layer 2 Head Layer 2, "to" với "take" "back" 
- Layer 2 Head 5 như Layer 0 Head 1
- Token "SOS" Nhận nhiều attention của các token khác (Layer 2 Head 4, 6) --> vì đây là decoder --> attention được dồn vào token này?


In [22]:
# Encoder Self-Attention
get_all_attention_maps("decoder", layers, heads, decoder_input_tokens, decoder_input_tokens, min(20, sentence_len))

alt.VConcatChart(...)

# Nhận xét giữa encoder (ngôn ngữ nguồn) và decoder (ngôn ngữ đích):
- Layer 1 Head 5,6,7 "luôn" với "always","take" --> attention giữa "luôn" với "always" (tần xuất) bổ nghĩa cho và "take" (động từ)

In [23]:
# Encoder Self-Attention
get_all_attention_maps("encoder-decoder", layers, heads, encoder_input_tokens, decoder_input_tokens, min(20, sentence_len))

alt.VConcatChart(...)